# Установка зависимостей
смотрите requirements.txt

In [2]:
import numpy as np
import pandas as pd
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn
from torchvision import transforms
import os
from torch.utils.data import Dataset
from tqdm import tqdm

import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

import timm

from dataset import *

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [18]:
DATA_DIR = "Train Dataset/videos" # директория с видео с разметкой
CLASSES = dict([(i, cl) for i, cl in enumerate(sorted(os.listdir('Train Dataset/videos')))])
SEED = 42

In [19]:
transform = transforms.Compose([
    transforms.Lambda(lambda x: x / 255.),
    transforms.Lambda(lambda x: x.permute(3, 0, 1, 2)),  # (T, C, H, W)
    transforms.Lambda(lambda x: torch.nn.functional.interpolate(x, (224, 224))),
])

full_dataset = VideoDataset(root=DATA_DIR, classes=CLASSES, num_frames=8, transform=transform)
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size],
                                                           torch.Generator().manual_seed(SEED))

In [20]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [23]:
model = torch.hub.load('facebookresearch/pytorchvideo', 'slow_r50', pretrained=True)
model.blocks[5].proj = nn.Linear(in_features=2048, out_features=24, bias=True) 

Downloading: "https://github.com/facebookresearch/pytorchvideo/zipball/main" to /Users/ivanmironov/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/pytorchvideo/model_zoo/kinetics/SLOW_8x8_R50.pyth" to /Users/ivanmironov/.cache/torch/hub/checkpoints/SLOW_8x8_R50.pyth
100%|████████████████████████████████████████████████████████████████████████████████████████| 248M/248M [00:04<00:00, 56.1MB/s]


In [28]:
model = model.to(device)

In [31]:
import wandb
wandb.login(key="416fa5ea55703d6fcbc4550f4fc08c7d5b672a86")
os.environ['WANDB_API_KEY'] = "416fa5ea55703d6fcbc4550f4fc08c7d5b672a86"

wandb: Currently logged in as: vanyamironov. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/ivanmironov/.netrc


In [32]:
wandb.init(project='digigtal_proriv',name='slow_r50')

In [33]:
criterion =  nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-5)

In [35]:
best_loss = 99
for epoch in range(10):  # loop over the dataset multiple times
    model.train()
    train_loss = 0.0
    for i, data in enumerate(tqdm(train_loader), 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        
        if torch.cuda.is_available():
            inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
    print(train_loss/len(train_dataset))
    wandb.log({"train_loss": train_loss/len(train_loader)}, step=epoch)

    valid_loss = 0
    valid_acc = 0
    valid_f1 = 0
    valid_precision = 0
    valid_recall = 0
    model.eval()
    for i, data in enumerate(tqdm(val_loader), 0):
        inputs, labels = data
        
        if torch.cuda.is_available():
            inputs, labels = inputs.to(device), labels.to(device)
        with torch.no_grad():
            target = model(inputs)
        
        loss = criterion(target, labels)
        valid_loss += loss.item()
        
        acc = accuracy_score(labels.cpu(), target.argmax(axis=1).cpu())
        valid_acc += acc
        
        f1 = f1_score(labels.cpu(), target.argmax(axis=1).cpu(), average="weighted")
        valid_f1 += f1
        
        precision = precision_score(labels.cpu(), target.argmax(axis=1).cpu(), average="weighted")
        valid_precision += precision

        recall = recall_score(labels.cpu(), target.argmax(axis=1).cpu(), average="weighted")
        valid_recall += recall
        
    print(valid_loss/len(val_dataset), valid_acc/len(val_loader))
    wandb.log({"val_loss": valid_loss/len(val_loader)}, step=epoch)
    wandb.log({"val_acc": valid_acc/len(val_loader)}, step=epoch)
    wandb.log({"val_f1": valid_f1/len(val_loader)}, step=epoch)
    wandb.log({"val_recall": valid_recall/len(val_loader)}, step=epoch)
    wandb.log({"val_precision": valid_precision/len(val_loader)}, step=epoch)
    torch.save(model.state_dict(), f'slow_r50_{epoch}')

  0%|▎                                                                                        | 1/283 [00:16<1:15:36, 16.09s/it]


KeyboardInterrupt: 